In [2]:
import arviz as az
import pymc as pm
from pymc.math import switch, ge
import pandas as pd
import numpy as np

# Loading Data, Step Function, and Deterministic Variables*

This example introduces data containers, tracking of deterministic variables, and shows how to recreate the BUGS step function in PyMC.

## Taste of Cheese

Adapted from [Unit 6: cheese.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit6/cheese.odc).

The link in the original .odc file is dead. I downloaded the data from [here](https://www3.nd.edu/~busiforc/handouts/Data%20and%20Stories/multicollinearity/Cheese%20Taste/Cheddar%20Cheese%20Data.html) and have a copy [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/cheese.csv).

As cheddar cheese matures, a variety of chemical processes take place. The taste of matured cheese is related to the concentration of several chemicals in the final product. In a study of cheddar cheese from the Latrobe Valley of Victoria, Australia, samples of cheese were analyzed for their chemical composition and were subjected to taste tests. Overall taste scores were obtained by combining the scores from several tasters.

Can the score be predicted well by the predictors: Acetic, H2S, and Lactic?

In [3]:
data = pd.read_csv("../data/cheese.csv", index_col=0)

In [4]:
data.head()

,taste,Acetic,H2S,Lactic
1,12.3,4.543,3.135,0.86
2,20.9,5.159,5.043,1.53
3,39.0,5.366,5.438,1.57
4,47.9,5.759,7.496,1.81
5,5.6,4.663,3.807,0.99


The main goal of this exercise is to practice loading the data and implementing a simple model.  Let's first specify the model structure, then write the code.  Since our goal is to model the relationship between chemical compounds and perceived taste, we can use a Bayesian linear regression model:

$$
\begin{align*}
\texttt{taste}_{i} &\sim \mathcal{N}(\mu_{i} , \sigma^{2})\\
\mu_{i} &= \beta_{0} + \beta_{1}\cdot\texttt{Acetic}_{i} + \beta_{2}\cdot\texttt{H2S}_{i} + \beta_{3}\cdot\texttt{Lactic}_{i}\\
\boldsymbol{\beta} &\sim \mathcal{N}(0,\sigma^{2} = 100000)\\
\sigma &= \frac{1}{\sqrt{\tau}}\\
\tau  &\sim \text{Gamma(0.001, 0.001)}\\ 
\end{align*}
$$

This model assumes that the taste score for each observation follows a normal distribution with mean $\mu_i$ and variance $\sigma^2$, where $\mu_i$ is a linear combination of the chemical predictors. We place vague normal priors on the regression coefficients and a weakly informative Gamma prior on the precision parameter $\tau$ (inverse variance).

It's often more practical to express the linear model in *matrix/vector form* rather than writing out each term separately:

$$
\boldsymbol{\eta} = \mathbf{X}\boldsymbol{\beta}
$$

Here $\boldsymbol{\eta}$ is the linear predictor, without adding any random noise.  If you're not familiar with the matrix notation for multiple linear regression, $\mathbf{X}$ and $\boldsymbol{\beta}$ are:

$$
\mathbf{X} = \begin{bmatrix}
1 & \texttt{Acetic}_1 & \texttt{H2S}_1 & \texttt{Lactic}_1 \\
1 & \texttt{Acetic}_2 & \texttt{H2S}_2 & \texttt{Lactic}_2 \\
\vdots & \vdots & \vdots & \vdots \\
1 & \texttt{Acetic}_n & \texttt{H2S}_n & \texttt{Lactic}_n
\end{bmatrix}
\text{ and }
\beta = 
\begin{bmatrix}
\text{Coefficient for Intercept}\\
\text{Coefficient for }\texttt{Acetic}\\
\text{Coefficient for }\texttt{H2S}\\
\text{Coefficient for }\texttt{Lactic}\\
\end{bmatrix}
$$

$\mathbf{X}$ is the design matrix: which contains all predictor columns along with a column of ones to account for the intercept $\beta_{0}$.  To make this clear in our code, we'll call it `X_aug`.

In [ ]:
X = data[["Acetic", "H2S", "Lactic"]].to_numpy()
# add intercept column to X
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
y = data["taste"].to_numpy()

In [6]:
with pm.Model() as m:
    # associate data with model (this makes prediction easier)
    X_data = pm.Data("X", X_aug)
    y_data = pm.Data("y", y)

    # priors
    beta = pm.Normal("beta", mu=0, sigma=1000, shape=X.shape[1] + 1)
    tau = pm.Gamma("tau", alpha=0.001, beta=0.001)
    sigma = pm.Deterministic("sigma", 1 / pm.math.sqrt(tau))

    mu = pm.math.dot(X_data, beta)

    # likelihood
    pm.Normal("taste_score", mu=mu, sigma=sigma, observed=y_data, shape=X_data.shape[0])

    trace = pm.sample(5000, target_accept=0.95)
    pm.sample_posterior_predictive(trace, extend_inferencedata=True)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, tau]


Output()

Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 12 seconds.
Sampling: [taste_score]


Output()

In [7]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],-29.098,20.297,-67.733,12.394,0.235,0.166,7438.0,9329.0,1.0
beta[1],0.367,4.605,-8.522,9.589,0.055,0.039,7017.0,8133.0,1.0
beta[2],3.895,1.304,1.324,6.453,0.014,0.010,9139.0,10809.0,1.0
beta[3],19.721,9.009,1.531,37.027,0.091,0.065,9940.0,11038.0,1.0
tau,0.010,0.003,0.005,0.015,0.000,0.000,10938.0,10771.0,1.0
sigma,10.437,1.512,7.800,13.503,0.015,0.011,10938.0,10771.0,1.0


In [8]:
y_pred = trace.posterior_predictive.stack(sample=("chain", "draw"))[
    "taste_score"
].values.T
az.r2_score(y, y_pred)

r2        0.576221
r2_std    0.076446
dtype: float64

Results are pretty close to OpenBUGS:

|         mean |     sd   |        MC_error |   val2.5pc | median  |   val97.5pc | start   | sample |        |
|--------------|----------|-----------------|------------|---------|-------------|---------|--------|--------|
| beta0        | -29.75   | 20.24           | 0.7889     | -70.06  | -29.75      | 11.11   | 1000   | 100001 |
| beta1        | 0.4576   | 4.6             | 0.189      | -8.716  | 0.4388      | 9.786   | 1000   | 100001 |
| beta2        | 3.906    | 1.291           | 0.02725    | 1.345   | 3.912       | 6.47    | 1000   | 100001 |
| beta3        | 19.79    | 8.893           | 0.2379     | 2.053   | 19.88       | 37.2    | 1000   | 100001 |
| tau          | 0.009777 | 0.002706        | 2.29E-05   | 0.00522 | 0.009528    | 0.01575 | 1000   | 100001 |

PyMC gives some warnings about the model unless we increase the ```target_accept``` parameter of ```pm.sample```, while BUGS doesn't. This is because PyMC uses more diagnostics to check if there are any problems with its exploration of the parameter space. Divergences indicate bias in the results. BUGS will happily run this model without reporting any problems, but it doesn't mean that there aren't any.

For further reading, check out [Diagnosing Biased Inference with Divergences](https://www.pymc.io/projects/examples/en/latest/diagnostics_and_criticism/Diagnosing_biased_Inference_with_Divergences.html).

## Stress, Diet, and Acids

Adapted from [Unit 6: stressacids.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit6/stressacids.odc).

In the study [Interrelationships Between Stress, Dietary Intake, and Plasma Ascorbic Acid During Pregnancy](https://vtechworks.lib.vt.edu/handle/10919/74486) conducted at the Virginia Polytechnic Institute and State University, the plasma ascorbic acid levels of pregnant women were compared for smokers versus non-smokers. Thirty-two women in the last three months of pregnancy, free of major health disorders, and ranging in age from 15 to 32 years were selected for the study. Prior to the collection of 20 ml of blood, the participants were told to avoid breakfast, forego their vitamin supplements, and avoid foods high in ascorbic acid content. From the blood samples, the plasma ascorbic acid values of each subject were determined in milligrams per 100 milliliters.

---

The purpose of this example in lectures was mostly just to show different ways to load data in BUGS. I'm not going to go into that too much, since there are a million ways to prepare your data in Python. In the next cell, I start with the data pasted from ```stressacids.odc```, then use list comprehensions to create one list for smokers and one for nonsmokers.

In [9]:
# fmt: off
plasma = [0.97, 0.72, 1.00, 0.81, 0.62, 1.32, 1.24, 0.99, 0.90, 0.74,
          0.88, 0.94, 1.06, 0.86, 0.85, 0.58, 0.57, 0.64, 0.98, 1.09,
          0.92, 0.78, 1.24, 1.18, 0.48, 0.71, 0.98, 0.68, 1.18, 1.36,
          0.78, 1.64]

smo = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
       1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]
# fmt: on

nonsmokers = [x for x, y in zip(plasma, smo) if y == 1]
smokers = [x for x, y in zip(plasma, smo) if y == 2]

### BUGS step function

I think this is the first time we've seen the BUGS step function.

BUGS defines the step function like this:

```step(e)``` is 1 if e >= 0; 0 otherwise.

Keep in mind that in PyMC, step functions are how they refer to the algorithms used for sampling, as in NUTS or Metropolis. Just different terminology.

We can recreate the BUGS step function with ```pm.math.switch()```:

```
pm.math.switch(e >= 0, 1, 0)

```

We should also probably use pm.math.ge for greater than or equal, as well, so:

```
pm.math.switch(ge(e,0), 1, 0)

```

### How do I track non-random variables in PyMC?

One nice thing about BUGS is you can easily track both deterministic and non-deterministic variables while sampling. For PyMC, you can wrap these in [```pm.Deterministic()```](https://docs.pymc.io/en/v3/api/model.html). Just make sure to use [```pm.math```](https://docs.pymc.io/en/v3/api/math.html) functions where possible.

In [10]:
with pm.Model() as m:
    # priors
    tau_nonsmokers = pm.Gamma("tau_nonsmokers", alpha=0.0001, beta=0.0001)
    sigma_nonsmokers = 1 / pm.math.sqrt(tau_nonsmokers)
    mu_nonsmokers = pm.Normal("mu_nonsmokers", mu=0, sigma=100)

    tau_smokers = pm.Gamma("tau_smokers", alpha=0.0001, beta=0.0001)
    sigma_smokers = 1 / pm.math.sqrt(tau_smokers)
    mu_smokers = pm.Normal("mu_smokers", mu=0, sigma=100)

    # likelihood
    pm.Normal(
        "nonsmokers_aa",
        mu=mu_nonsmokers,
        sigma=sigma_nonsmokers,
        observed=nonsmokers,
    )
    pm.Normal("smokers_aa", mu=mu_smokers, sigma=sigma_smokers, observed=smokers)

    testmu = pm.Deterministic("test_mu", switch(ge(mu_smokers, mu_nonsmokers), 1, 0))
    r = pm.Deterministic("prec_ratio", tau_nonsmokers / tau_smokers)

    trace = pm.sample(5000)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau_nonsmokers, mu_nonsmokers, tau_smokers, mu_smokers]


Output()

Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 2 seconds.


In [11]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu_nonsmokers,0.912,0.045,0.820,1.000,0.000,0.000,15233.0,12039.0,1.0
mu_smokers,0.975,0.163,0.643,1.293,0.001,0.001,14256.0,10128.0,1.0
tau_nonsmokers,22.570,6.596,10.520,35.683,0.053,0.038,14810.0,12597.0,1.0
tau_smokers,6.521,3.530,0.761,13.378,0.029,0.021,13420.0,11155.0,1.0
test_mu,0.665,0.472,0.000,1.000,0.004,0.003,16922.0,16922.0,1.0
prec_ratio,4.896,4.343,0.652,12.053,0.043,0.031,13700.0,11313.0,1.0


In [12]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

Last updated: Sun Mar 09 2025

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.29.0

pytensor: 2.26.4

numpy : 1.26.4
arviz : 0.20.0
pymc  : 5.19.1
pandas: 2.2.3

